In [1]:
import torch
import torch.nn as nn

In [2]:
import torch
import torchvision.models as models

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [4]:
import pandas as pd
import torch
import torchaudio
from torch.utils.data import Dataset


class DCaseDataset(Dataset):
    """
    Dataloader for DCase dataset
    Structure of the class is taken from:
    https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/audio_classifier_tutorial.ipynb
    """

    labelind2name = {
        0: "airport",
        1: "bus",
        2: "metro",
        3: "metro_station",
        4: "park",
        5: "public_square",
        6: "shopping_mall",
        7: "street_pedestrian",
        8: "street_traffic",
        9: "tram",
    }
    name2labelind = {
        "airport": 0,
        "bus": 1,
        "metro": 2,
        "metro_station": 3,
        "park": 4,
        "public_square": 5,
        "shopping_mall": 6,
        "street_pedestrian": 7,
        "street_traffic": 8,
        "tram": 9,
    }

    def __init__(self, root_dir, csv_path, indices):
        """

        :param root_dir:
        :param split:
        """

        # Open csv files
        self.root_dir = root_dir
        #if split == "train":
        #    csv_path = root_dir + "/evaluation_setup/fold1_train.csv"
        #    meta_path = root_dir + "/meta.csv"
        #elif split == "val":
        #    csv_path = root_dir + "/evaluation_setup/fold1_evaluate.csv"
        #    meta_path = root_dir + "/meta.csv"
        #elif split == "test":
        #    csv_path = root_dir + "/evaluation_setup/fold1_test.csv"
        #    meta_path = None
        #else:
        #    raise ValueError("Split not implemented")
        
        csvData = pd.read_csv(csv_path, sep="\t")
        csvData["label"] = csvData.apply(lambda x: x["filename"].split("-")[0], axis=1)
        #metaData = pd.read_csv(meta_path, sep="\t") if meta_path is not None else None

        # In test mode, just get file list
        #if split == "test":
        #    self.file_names = []
        #    for i in range(0, len(csvData)):
        #        self.file_names.append(csvData.iloc[i, 0])
        #    return

        # Lists of file names and labels
        self.file_names, self.labels = [], []
        for i in indices:
            self.file_names.append(csvData.iloc[i, 0])
            self.labels.append(csvData.iloc[i, 1])

        # Device for each audio file
        # self.devices = {}
        # for i in range(0, len(metaData)):
        #    self.devices[metaData.iloc[i, 0]] = metaData.iloc[i, 3]

        # Transform class name to index
        self.labels = [self.name2labelind[name] for name in self.labels]

    def __getitem__(self, index):
        """

        :param index:
        :return:
        """

        # Load data
        filepath = self.root_dir + self.file_names[index]
        sound, sfreq = torchaudio.load(filepath, normalize=True)
        assert sound.shape[0] == 1, "Expected mono channel"
        sound = torch.mean(sound, dim=0)
        assert sfreq == 44100, "Expected sampling rate of 44.1 kHz"

        # Remove last samples if longer than expected
        if sound.shape[-1] >= 441000:
            sound = sound[:441000]

        return (
                sound,
                self.labels[index],
                self.file_names[index]
            )

    def __len__(self):
        return len(self.file_names)


In [5]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader
from sp4asc.models.cnns import LogMelSpectrogram
from sp4asc.models import get_net
from sp4asc.training import TrainingManager


In [6]:
root_dir = "/users/eleves-b/2019/maxime.bonnin/MAP583/TAU_SMALL_DATASET/test/"
csv_path = "/users/eleves-b/2019/maxime.bonnin/MAP583/TAU_SMALL_DATASET/fold1_small_train.csv"

N = len(os.listdir(root_dir))
train_indices, test_indices = train_test_split(np.arange(0,N), test_size=0.33)

train_dataset = DCaseDataset(root_dir, csv_path, train_indices)
test_dataset = DCaseDataset(root_dir, csv_path, test_indices)

In [26]:
config = {
    "batchsize": 32,
    "num_workers": 4,
    "reload": False,
    "net": "Cnn6_60k",
    "dropout": 0.2,
    "specAugment": [128, 2, 16, 2],
    "lr": 1e-3,
    "eta_min": 1e-5,
    "max_epoch": 100,
    "weight_decay": 1e-5,
    "mixup_alpha": 0.2,
    "out_dir": "./trained_models/",
}

In [27]:
loader_train = DataLoader(
        train_dataset,
        batch_size=config["batchsize"],
        shuffle=True,
        pin_memory=True,
        num_workers=config["num_workers"],
        drop_last=True,
    )
loader_test = DataLoader(
    test_dataset,
    batch_size=config["batchsize"],
    shuffle=False,
    pin_memory=True,
    num_workers=config["num_workers"],
    drop_last=False,
)

In [28]:
spectrogram = LogMelSpectrogram()
net = get_net[config["net"]](
    config["dropout"],
    config["specAugment"],
)
#net.load(pathModel, map_location=torch.device("cpu"))
#
#for param in net.parameters():
#    param.requires_grad = False
#    
#for param in net.fc2.parameters():
#    param.requires_grad = True

print("\n\nNet at training time")
print("Nb. of parameters at training time: ", count_parameters(net) / 1e3, "k")




Net at training time
Nb. of parameters at training time:  62.922 k


In [29]:
# make sure to load the pretrained model
root = "/users/eleves-b/2019/maxime.bonnin/"
pathModel = root + "MAP583/trained_models/test1"

In [ ]:
# ---
path2log = config["out_dir"] + "test2"
os.makedirs(path2log, exist_ok=True)

optim = torch.optim.AdamW(
    [
        {"params": net.parameters()},
    ],
    lr=config["lr"],
    weight_decay=config["weight_decay"],
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optim,
    config["max_epoch"],
    eta_min=config["eta_min"],
)

# --- Training
mng = TrainingManager(
    net,
    spectrogram,
    loader_train,
    loader_test,
    optim,
    scheduler,
    config,
    path2log,
    device="cuda"
)
mng.path_to_ckpt = pathModel + "/ckpt.pth"
mng.load_state()
mng.current_epoch = 0
mng.path_to_ckpt = path2log + "/ckpt.pth"
mng.max_epoch = 100

mng.train()


Training: 0/100 epochs


       0%|                                                  | 0/62 [00:00<?, ?it/s]


Epoch: 0 : loss = 1.080 - acc1 = 68.750

Epoch: 0 : loss = 1.185 - acc1 = 63.672

Epoch: 0 : loss = 1.178 - acc1 = 63.333

Epoch: 0 : loss = 1.176 - acc1 = 63.256

Test:


       0%|                                                  | 0/31 [00:00<?, ?it/s]


Epoch: 0 : loss = 0.853 - acc1 = 64.688

Epoch: 0 : loss = 0.822 - acc1 = 67.344

Epoch: 0 : loss = 0.853 - acc1 = 65.729

Epoch: 0 : loss = 0.861 - acc1 = 65.816

Training: 1/100 epochs


...

       0%|                                                  | 0/62 [00:00<?, ?it/s]


Epoch: 57 : loss = 1.126 - acc1 = 64.375

Epoch: 57 : loss = 1.198 - acc1 = 64.375

Epoch: 57 : loss = 1.248 - acc1 = 62.448

Epoch: 57 : loss = 1.245 - acc1 = 62.651

Test:


       0%|                                                  | 0/31 [00:00<?, ?it/s]


Epoch: 57 : loss = 0.968 - acc1 = 64.062

Epoch: 57 : loss = 0.935 - acc1 = 64.844

Epoch: 57 : loss = 0.961 - acc1 = 63.854

Epoch: 57 : loss = 0.972 - acc1 = 63.571

Training: 58/100 epochs


       0%|                                                  | 0/62 [00:00<?, ?it/s]

In [10]:
small_densenet121 = models.densenet121(pretrained=True)
print("small DenNet:", count_parameters(small_densenet121))

small DenNet: 7978856


In [15]:
for params in small_densenet121.parameters():
    params.requires_grad = False

small_densenet121.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
small_densenet121.classifier = nn.Linear(in_features=1024, out_features=10, bias=True)
print("small DenNet:", count_parameters(small_densenet121))
small_densenet121 = small_densenet121.cuda()#.to("cpu")

small DenNet: 13386


In [18]:
# ---
from sp4asc.training import TrainingManager

spectrogram = LogMelSpectrogram()

path2log = config["out_dir"] + "test1"
os.makedirs(path2log, exist_ok=True)

optim = torch.optim.AdamW(
    [
        {"params": small_densenet121.parameters()},
    ],
    lr=config["lr"],
    weight_decay=config["weight_decay"],
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optim,
    config["max_epoch"],
    eta_min=config["eta_min"],
)

# --- Training
mng = TrainingManager(
    small_densenet121,
    spectrogram,
    loader_train,
    loader_test,
    optim,
    scheduler,
    config,
    path2log,
    device="cpu"
)
mng.train()


Training: 0/200 epochs


       0%|                                                  | 0/62 [00:00<?, ?it/s]


Epoch: 0 : loss = 2.351 - acc1 = 11.719

Epoch: 0 : loss = 2.308 - acc1 = 14.141


KeyboardInterrupt: 